In [ ]:
import numpy as np
from flask import Flask, request, render_template, url_for
import os
import plotly.express as px
import pandas as pd
import joblib
import pandas as pd
import numpy as np
import seaborn as sns
from scipy import stats 
import matplotlib.pyplot as plt
import hvplot.pandas
import holoviews as hv
from holoviews import opts
hv.extension('bokeh')
from io import BytesIO
import base64
import plotly.graph_objs as go
import plotly.io as pio
from plotly.subplots import make_subplots

app = Flask(__name__)


# Load the model for Mortgage_loan_application route
# Load the dataset
try:
    loan_data = pd.read_csv('Home_Loan_Approval_System/loan_sanction_train_14(2).csv')
    test_data = pd.read_csv('Home_Loan_Approval_System/loan_sanction_test14(1).csv')

except Exception as e:
    print("An error occurred while loading or preprocessing the dataset:", str(e))
    loan_data = None
    test_data = None

# Load the machine learning model
try:
    with open('Home_Loan_Approval_System/model.pkl', 'rb') as model_file:
        model = joblib.load(model_file)
except FileNotFoundError as e:
    print("Model file not found:", e)
    model = None
except Exception as e:
    print("An error occurred while loading the model:", e)
    model = None


try:
    # Load the machine learning model
    with open('model7.pkl', 'rb') as model_file2:
        model2 = joblib.load(model_file2)
except FileNotFoundError:
    print("Model file not found.")
    model2 = None
except Exception as e:
    print("An error occurred while loading the model:", str(e))
    model2 = None
    
try:
    df = pd.read_csv('lending_club_loan_two.csv')
    if df.empty:
        print("Dataset is empty.")
        df = None
except FileNotFoundError:
    print("Dataset file not found.")
    df = None
except Exception as e:
    print("An error occurred while loading the dataset:", str(e))
    df = None    

# Define preprocessdata function for Mortgage_loan_application route
def preprocessdatamort(Married, Dependents, Education, Self_Employed, LoanAmount, Loan_Amount_Term, Credit_History,
                   ApplicantIncome, CoapplicantIncome, Gender, Property_Area):
    # Preprocessing logic for Mortgage_loan_application route
    try:
        Loan_Amount_Term = float(Loan_Amount_Term)
        
        # Validate and convert Credit_History to an integer value (0 or 1)
        if Credit_History not in ['0', '1']:
            raise ValueError("Invalid value for Credit_History. Must be '0' or '1'.")
        Credit_History = int(Credit_History)
    
        test_data_df2 = pd.DataFrame({
            'Married': [1 if Married == 'Yes' else 0],
            'Dependents': [Dependents],
            'Education': [1 if Education == 'Graduate' else 0],
            'Self_Employed': [1 if Self_Employed == 'Yes' else 0],
            'LoanAmount': [float(LoanAmount)],
            'Loan_Amount_Term': [Loan_Amount_Term],
            'Credit_History': [Credit_History],
            'Total_Income': [float(ApplicantIncome + CoapplicantIncome)],
            'Gender_Male': [1 if Gender == 'Male' else 0],
            'PA_Semiurban': [1 if Property_Area == 'Semiurban' else 0],
            'PA_Urban': [1 if Property_Area == 'Urban' else 0]
        })
        
        
        print("Test Data:")
        print(test_data_df2)  # Print out the test data for debugging
        
        predict = model.predict(test_data_df2)
        print("Prediction Value:", predict)


        predict = model.predict(test_data_df2)
        if predict[0] == 1:
            return "Yes, You are eligible for loan"
        elif predict[0] == 2:
            return "No, You are not eligible for loan"
        else:
            return "Unknown"
        return predict[0]
    except Exception as e:
        print("An error occurred during data preprocessing or prediction:", str(e))
        return None

def preprocessdatagen(loan_amnt, term, int_rate, installment, grade, sub_grade, emp_title, emp_length, annual_inc, verification_status, dti, earliest_cr_line, 
                                    open_acc, revol_bal, revol_util,
                                    total_acc, application_type, address_encoded):
    
    # Preprocessing logic for gen_application route
    try:
        # Convert earliest_cr_line to days since a reference date
        reference_date = pd.to_datetime('1900-01-01')
        earliest_cr_line_days = (pd.to_datetime(earliest_cr_line) - reference_date).days
        
        test_data1 = {
            "loan_amnt": [float(loan_amnt)],
            "term": [int(term)],
            "int_rate": [float(int_rate)],
            "installment": [float(installment)],
            "grade": [int(grade)],
            "sub_grade": [int(sub_grade)],
            "emp_title": [int(emp_title)],
            "emp_length": [int(emp_length)],
            "annual_inc": [float(annual_inc)],
            "verification_status": [int(verification_status)],
            "dti":[float(dti)],
            "earliest_cr_line": [earliest_cr_line_days],  # Use the numeric representation
            "open_acc": [int(open_acc)],
            "revol_bal": [float(revol_bal)],
            "revol_util": [float(revol_util)],
            "total_acc": [float(total_acc)],
            "application_type": [int(application_type)],
            "address_encoded": [int(address_encoded)]
        }
        
        test_data_df1 = pd.DataFrame(test_data1)
        
        print("Test Data:")
        print(test_data_df1)  # Print out the test data for debugging
        
        prediction1 = model2.predict(test_data_df1)
        print("Prediction Value:", prediction1)

        prediction = model2.predict(test_data_df1)
        if prediction1[0] == 1:
            return "Yes, You are eligible for a loan"
        elif prediction1[0] == 0:
            return "No, You are not eligible for a loan"
        else:
            return "Unknown"
    except Exception as e:
        print("An error occurred during data preprocessing or prediction:", str(e))
        return None


imageFolder = os.path.join('static', 'images')
# Create the directory if it doesn't exist
if not os.path.exists(imageFolder):
    os.makedirs(imageFolder)

app.config['UPLOAD_FOLDER'] = imageFolder


@app.route('/')
def home():
    # List all images in the directory
    try:
        imageList = os.listdir(imageFolder)
    except FileNotFoundError:
        imageList = []

    return render_template('home.php', user_image=imageList)
    

@app.route('/Mortgage_loan_application', methods=["GET", "POST"])
def Mortgage_loan_application():
    if request.method == "POST":
        # Extract form data
        Married = request.form.get('Married')
        Dependents = request.form.get('Dependents')
        Education = request.form.get('Education')
        Self_Employed = request.form.get('Self_Employed')
        LoanAmount = float(request.form.get('LoanAmount'))
        Loan_Amount_Term = request.form.get('Loan_Amount_Term')
        Credit_History = request.form.get('Credit_History')
        ApplicantIncome = float(request.form.get('ApplicantIncome'))
        CoapplicantIncome = float(request.form.get('CoapplicantIncome'))
        Gender = request.form.get('Gender')
        Property_Area = request.form.get('Property_Area')
        
        # Pass form data to preprocessdata function
        predict = preprocessdatamort(Married, Dependents, Education, Self_Employed, LoanAmount, Loan_Amount_Term, Credit_History, 
                                    ApplicantIncome, CoapplicantIncome, Gender, Property_Area)
        
        print("Prediction is: ", predict)

        # Check if prediction is not None
        if predict is not None:
            # Render template with prediction and form data
            return render_template('Mortgage_loan_application.html', Married=Married, Dependents=Dependents, Education=Education,
                                   Self_Employed=Self_Employed, LoanAmount=LoanAmount, Loan_Amount_Term=Loan_Amount_Term, Credit_History=Credit_History, 
                                   ApplicantIncome=ApplicantIncome, CoapplicantIncome=CoapplicantIncome, Gender=Gender, Property_Area=Property_Area, predict=predict)
        else:
            # Return an error message if prediction is None
            return "Prediction failed. Please check your input data and try again."
        
    # If it's not a POST request, render the template without any form data
    return render_template('Mortgage_loan_application.html')



@app.route('/gen_application', methods=['GET', 'POST'])
def gen_application():
    grade = {'A':1, 'B':2, 'C':3, 'D':4, 'E':5, 'F':6, 'G':7}
    sub_grade = {
        'A1':1, 'B1':2, 'C1':3, 'D1':4, 'E1':5, 'F1':6, 'G1':7,
        'A2':8, 'B2':9, 'C2':10, 'D2':11, 'E2':12, 'F2':13, 'G2':14,
        'A3':15, 'B3':16, 'C3':17, 'D3':18, 'E3':19, 'F3':20, 'G3':21,
        'A4':22, 'B4':23, 'C4':24, 'D4':25, 'E4':26, 'F4':27, 'G4':28,
        'A5':29, 'B5':30, 'C5':31, 'D5':32, 'E5':33, 'F5':34, 'G5':35
    }
    emp_length = {'Less than 1 year':0, '1 year':1, '2 years':2, '3 years':3, '4 years':4, '5 years':5, '6 years':6, '7 years':7,
                 '8 years':8, '9 years':9, '10+ years':10}
    emp_title = {
    'Teacher': 1,
    'Manager': 2,
    'Registered Nurse': 3,
    'RN': 4,
    'Supervisor': 5,
    'Sales': 6,
    'Project Manager': 7,
    'Owner': 8,
    'Driver': 9,
    'Office Manager': 10,
    'manager': 11,
    'Director': 12,
    'General Manager': 13,
    'Engineer': 14,
    'teacher': 15,
    'driver': 16,
    'Vice President': 17,
    'Operations Manager': 18,
    'Administrative Assistant': 19,
    'Accountant': 20
}

    if request.method == 'POST':
        loan_amnt = float(request.form.get('loan_amnt'))
        term = float(request.form.get('term'))
        int_rate = float(request.form.get('int_rate'))
        installment = int(request.form.get('installment'))
        grade = int(request.form.get('grade'))
        sub_grade = int(request.form.get('sub_grade'))
        emp_title = int(request.form.get('emp_title'))
        emp_length = int(request.form.get('emp_length'))
        annual_inc = float(request.form.get('annual_inc'))
        verification_status = int(request.form.get('verification_status'))
        dti = float(request.form.get('dti'))
        earliest_cr_line = pd.to_datetime(request.form.get('earliest_cr_line'))
        open_acc = int(request.form.get('open_acc'))
        revol_bal = float(request.form.get('revol_bal'))
        revol_util = float(request.form.get('revol_util'))
        total_acc = float(request.form.get('total_acc'))
        application_type = int(request.form.get('application_type'))
        address_encoded = int(request.form.get('address_encoded'))
        
        print("Received form data:")
        print("loan_amnt:", loan_amnt)
        print("term:", term)
        print("int_rate:", int_rate)
        print("installment:", installment)
        print("grade:", grade)
        print("sub_grade:", sub_grade)
        print("emp_title:", emp_title)
        print("emp_length:", emp_length)
        print("annual_inc:", annual_inc)
        print("verification_status:", verification_status)
        print("dti:", dti)
        print("earliest_cr_line:", earliest_cr_line)
        print("Open_acc:", open_acc)
        print("Revol_Bal:", revol_bal)
        print("Revol_util:", revol_util)
        print("Total_acc:", total_acc)
        print("application_type:", application_type)
        print("address_encoded:", address_encoded)
        
       
        prediction = preprocessdatagen(loan_amnt, term, int_rate, installment, grade, sub_grade, emp_title, emp_length, annual_inc, verification_status, dti, earliest_cr_line,
                                    open_acc, revol_bal, revol_util,
                                    total_acc, application_type, address_encoded)
        
        print('Prediction', prediction)

        return render_template('gen_application.html', loan_amnt=loan_amnt, term=term, int_rate=int_rate, installment=installment, grade=grade, sub_grade=sub_grade,emp_title=emp_title,
                               emp_length=emp_length, annual_inc=annual_inc, verification_status=verification_status, dti=dti, earliest_cr_line=earliest_cr_line, open_acc=open_acc,
                               revol_bal=revol_bal, revol_util=revol_util,
                               total_acc=total_acc,
                               application_type=application_type,address_encoded=address_encoded, prediction=prediction)

    return render_template('gen_application.html')

@app.route('/combined_plots')
def combined_plots():
    # Load data
    data = pd.read_csv("lending_club_loan_two.csv")

    # Preprocess data
    data['issue_d'] = pd.to_datetime(data['issue_d'], format='%b-%Y')

    # Create histograms and bar charts
    installment = data.hvplot.hist(
        y='installment', by='loan_status', subplots=False, 
        width=350, height=400, bins=50, alpha=0.4, 
        title="Installment by Loan Status", 
        xlabel='Installment', ylabel='Counts', legend='top'
    )

    loan_amnt = data.hvplot.hist(
        y='loan_amnt', by='loan_status', subplots=False, 
        width=350, height=400, bins=30, alpha=0.4, 
        title="Loan Amount by Loan Status", 
        xlabel='Loan Amount', ylabel='Counts', legend='top'
    )


    # Combine plots into a layout
    combined_plots = (installment + loan_amnt).cols(2)

    # Save the combined plot as an HTML file
    hv.save(combined_plots, 'templates/combined_plots.html', combined_plots=combined_plots)
    
    # Render your template and pass the combined_plots_html variable
    return render_template('combined_plots.html')
    

@app.route('/combined_plots1')
def combined_plots1():
    # Load data
    data = pd.read_csv("lending_club_loan_two.csv")

    # Create histograms and bar charts
    loan_status_counts_plot = data['loan_status'].value_counts().hvplot.bar(
        title="Loan Status Counts", xlabel='Loan Status', ylabel='Count', 
        width=500, height=350
    )

    installment_plot = data.hvplot.hist(
        y='installment', by='loan_status', subplots=False, 
        width=350, height=400, bins=50, alpha=0.4, 
        title="Installment by Loan Status", 
        xlabel='Installment', ylabel='Counts', legend='top'
    )
    loan_amnt_plot = data.hvplot.hist(
        y='loan_amnt', by='loan_status', subplots=False, 
        width=350, height=400, bins=30, alpha=0.4, 
        title="Loan Amount by Loan Status", 
        xlabel='Loan Amount', ylabel='Counts', legend='top'
    )

    fully_paid_grade_plot = data.loc[data['loan_status']=='Fully Paid', 'grade'].value_counts().hvplot.bar(
        title="Fully Paid Loans by Grade", xlabel='Grades', ylabel='Count',
        width=500, height=450, legend_cols=2, legend_position='top_right', xrotation=90
    )

    charged_off_grade_plot = data.loc[data['loan_status']=='Charged Off', 'grade'].value_counts().hvplot.bar(
        title="Charged Off Loans by Grade", xlabel='Grades', ylabel='Count',
        width=500, height=400, legend_cols=2, legend_position='top_right', xrotation=90
    )

    fully_paid_sub_grade_plot = data.loc[data['loan_status']=='Fully Paid', 'sub_grade'].value_counts().hvplot.bar(
        title="Fully Paid Loans by Sub Grade", xlabel='Grades', ylabel='Count',
        width=500, height=400, legend_cols=2, legend_position='top_right', xrotation=90
    )

    charged_off_sub_grade_plot = data.loc[data['loan_status']=='Charged Off', 'sub_grade'].value_counts().hvplot.bar(
        title="Charged Off Loans by Sub Grade", xlabel='Grades', ylabel='Count',
        width=500, height=400, legend_cols=2, legend_position='top_right', xrotation=90
    )

    home_ownership_count_plot = (
        data.groupby(['home_ownership', 'loan_status']).size().reset_index(name='count')
        .hvplot.bar(x='home_ownership', y='count', by='loan_status', stacked=True, 
                    title="Loan Status by Home Ownership", xlabel='Home Ownership', 
                    ylabel='Count', width=350, height=350)
    )

    home_ownership_plot = data.home_ownership.value_counts().hvplot.bar(
        title="Loan Status by Home Ownership", xlabel='Home Ownership', ylabel='Count', 
        width=350, height=350, legend='top'
    )

    int_rate_plot = data.hvplot.hist(
        y='int_rate', by='loan_status', alpha=0.3, width=350, height=400,
        title="Loan Status by Interest Rate", xlabel='Interest Rate', ylabel='Loans Counts', 
        legend='top'
    )

    annual_inc_plot = data.hvplot.hist(
        y='annual_inc', by='loan_status', bins=50, alpha=0.3, width=350, height=400,
        title="Loan Status by Annual Income", xlabel='Annual Income', ylabel='Loans Counts', 
        legend='top'
    ).opts(xrotation=45)

    # Combine plots into a layout
    combined_plots1 = (loan_status_counts_plot + installment_plot + loan_amnt_plot +
                      fully_paid_grade_plot + charged_off_grade_plot +
                      fully_paid_sub_grade_plot + charged_off_sub_grade_plot +
                      home_ownership_count_plot + home_ownership_plot +
                      int_rate_plot + annual_inc_plot).cols(2)
    
    # Save the combined plot as an HTML file
    hv.save(combined_plots1, 'templates/combined_plots1.html')

    # Pass the combined_plots1 variable to the template
    return render_template('combined_plots1.html')



@app.route('/generated_graphs')
def generated_graphs():
    # Read the dataset
    loan_data = pd.read_csv('Home_Loan_Approval_System/loan_sanction_train_14(2).csv')

    html_content = ""  # Initialize HTML content string

    #Data preparation
    loan_data = loan_data.drop('Loan_ID',axis = 1)

    # Analysis - Impact of Gender, Education, Self Employed, and Credit History on Loan Status
    for col in ['Education', 'Gender', 'Self_Employed', 'Credit_History']:
        col_label = col.replace("_", " ")

        # Group data by column and loan status, count occurrences, and reshape it
        df = loan_data.groupby([col, 'Loan_Status']).size().unstack(fill_value=0).reset_index()

        fig = go.Figure()
        fig.add_trace(go.Bar(x=df[col], y=df['Y'], marker_color=['#008744','#0057e7'], text=df['Y'], 
                             textposition='auto', name='Loan Approved'))
        
        fig.update_layout(yaxis_range=[0,350])  
        fig.update_yaxes(showgrid=True)
        fig.update_xaxes(showgrid=True) 

        fig.update_layout(height=400,width=800,
              title=f'<b>{col_label} Vs Loan Status</b>',
              font_family="Times New Roman",
              title_font_family="Times New Roman",
              title_font_color="#000000",
              title_font_size=20,
              title_xanchor="center",
              title_x=0.5,
              xaxis_title=f"<b>{col_label}</b>",
              yaxis_title="<b>Count</b>",
              legend_title='<b>Loan Status</b>',
              legend_title_font_color="#000000",
              plot_bgcolor ='#ffffff'
          )

        
        # Generate HTML content for the plotly figure
        html_content += pio.to_html(fig, full_html=False)


    # Analysis - Impact of Property Area on Loan Approval
    pa_loan_df = loan_data[['Property_Area', 'LoanAmount']].groupby(['Property_Area']).agg({'Property_Area':'count', 'LoanAmount':'sum'})
    pa_loan_df.columns = ['Loan_Count', 'Loan_Amount']
    pa_loan_df.reset_index(inplace=True)

    fig = make_subplots(rows=1, cols=2, specs=[[{'type':'xy'}, {'type':'domain'}]],
                        subplot_titles=['<b>Loan Count</b>', '<b>Loan Amount</b>'])

    fig.add_trace(go.Bar(x=pa_loan_df['Property_Area'], y=pa_loan_df['Loan_Count'], text=pa_loan_df['Loan_Count'], 
                         marker_color=['#7bc043', '#0392cf', '#f37736']), row=1, col=1)
    fig.add_trace(go.Pie(labels=pa_loan_df['Property_Area'], values=pa_loan_df['Loan_Amount'], name='Loan_Amount', 
                         textinfo='percent+label+value', marker_colors=['#7bc043', '#0392cf', '#f37736']), row=1, col=2)

    fig.update_layout(
        title_text="<b>Property Area wise Loan Distribution</b>",
        plot_bgcolor='#ffffff'
    )

    # Generate HTML content for the plotly figure
    html_content += pio.to_html(fig, full_html=False)

    # Save HTML content to a file
    file_path = "templates/generated_graphs.html"
    with open(file_path, "w") as file:
        file.write(html_content)

    # Open the HTML file in the browser
    os.system("start " + file_path)

    return "Generated graphs have been saved and opened in the browser."



if __name__ == "__main__":
    app.run()


 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on http://127.0.0.1:5000
Press CTRL+C to quit
127.0.0.1 - - [21/Apr/2024 08:48:36] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [21/Apr/2024 08:48:36] "GET /static/total.png HTTP/1.1" 404 -
127.0.0.1 - - [21/Apr/2024 08:48:36] "GET /static/IntAnnual.png HTTP/1.1" 404 -
127.0.0.1 - - [21/Apr/2024 08:48:36] "GET /templates/static/dti.png HTTP/1.1" 404 -
127.0.0.1 - - [21/Apr/2024 08:48:36] "GET /static/roc1.png HTTP/1.1" 404 -
127.0.0.1 - - [21/Apr/2024 08:48:42] "GET /static/images/logo.svg HTTP/1.1" 404 -
